The code in the next cell is used to find the features that distinguish motion from no motion the best. 

In [ ]:
from itertools import combinations

def _f1(cm, idx):
    tp = cm[idx, idx]
    fn = cm[idx, :].sum() - tp
    fp = cm[:, idx].sum() - tp
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

def print_combined_metrics(cm, labels):
    """
    Print precision, recall, F1 for each class + return macro F1 for FALL and MOTION only.
    """
    print("\n📊 Class-wise Precision, Recall, F1:")
    for i, label in enumerate(labels):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        print(f"{label}: Precision={precision:.2f}, Recall={recall:.2f}, F1={f1:.2f}")

    macro_f1 = 0
    try:
        fall_idx = list(labels).index("FALL")
        motion_idx = list(labels).index("MOTION")
        fall_f1 = _f1(cm, fall_idx)
        motion_f1 = _f1(cm, motion_idx)
        macro_f1 = (fall_f1 + motion_f1) / 2
        print(f"\n⭐ Macro F1 (FALL vs MOTION): {macro_f1:.4f}")
    except ValueError:
        print("\n⚠️ FALL or MOTION not found in labels. Skipping macro F1 calc.")

    return macro_f1

def main():
    """
    Run LOSO CV on top feature combos + merged hybrids.
    Print confusion matrices and metrics, and identify the best-performing combo.
    """
    original_combos = [
        ['acc_mag_iqr', 'acc_impact_peak_val', 'tilt_angle_mean'],
        ['acc_mag_range', 'acc_mag_rms', 'acc_x_range'],
        ['acc_mag_rms', 'acc_num_peaks', 'gyro_mag_median'],
        ['acc_mag_rms', 'sma', 'acc_y_range'],
        ['acc_mag_rms', 'acc_y_range', 'acc_mag_p75'],
        ['acc_mag_rms', 'acc_x_range', 'acc_y_range'],
        ['acc_num_peaks', 'tilt_angle_mean', 'acc_x_range']
    ]

    # Generate merged combos (up to 7 features per combo)
    hybrid_combos = []
    for c1, c2 in combinations(original_combos, 2):
        merged = sorted(set(c1) | set(c2))
        if len(merged) <= 7:
            hybrid_combos.append(merged)

    all_combos = original_combos + hybrid_combos
    print(f"\n🔢 Total combinations to evaluate: {len(all_combos)}")

    best_params = {'learning_rate': 0.01, 'max_iter': 200, 'max_leaf_nodes': 31}

    best_score = -1
    best_combo = None
    best_cm = None
    best_labels = None

    for i, selected_features in enumerate(all_combos, 1):
        print("="*60)
        print(f"\n🔍 Combo {i}/{len(all_combos)}: {selected_features}")
        print("-"*60)

        session_results, le, labels = loso_cv_print_each(df_features, selected_features, best_params)

        combined_cm = np.zeros_like(session_results[0][2])
        for _, _, cm in session_results:
            combined_cm += cm

        print("\nCombined Confusion Matrix:")
        print(combined_cm)
        plot_row_normalized_confusion_matrix(combined_cm, labels, f"Combo {i} Confusion Matrix (Row-Normalized)")
        macro_f1 = print_combined_metrics(combined_cm, labels)

        # Track best
        if macro_f1 > best_score:
            best_score = macro_f1
            best_combo = selected_features
            best_cm = combined_cm
            best_labels = labels

    # Final best result
    print("\n" + "="*60)
    print("🏆 BEST COMBINATION FOUND:")
    print("Features:", best_combo)
    print(f"Macro F1 Score (Fall vs Motion): {best_score:.4f}")
    print("Combined Confusion Matrix:")
    print(best_cm)
    plot_row_normalized_confusion_matrix(best_cm, best_labels, "Best Combo Confusion Matrix (Row-Normalized)")
    print_combined_metrics(best_cm, best_labels)


In [ ]:
df_raw = read_data("training_data.csv")
df_features = extract_advanced_features_from_timeseries(df_raw, window_size=2, step_size=0.25, do_fft=True)
main()